In [ ]:
import os
import yt_dlp
from moviepy.editor import VideoFileClip
import speech_recognition as sr
from pydub import AudioSegment  # Make sure to import AudioSegment

def download_video(url, output_path="downloaded_video.mp4"):
    ydl_opts = {'outtmpl': output_path, 'format': 'best'}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def extract_audio(video_path, audio_path="audio.wav"):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    return audio_path

def split_audio(audio_path, chunk_length_ms=60000):
    audio = AudioSegment.from_wav(audio_path)
    chunks = []
    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i+chunk_length_ms]
        chunk_name = f"chunk{i//chunk_length_ms}.wav"
        chunk.export(chunk_name, format="wav")
        chunks.append(chunk_name)
    return chunks

def recognize_audio_chunks(chunks):
    recognizer = sr.Recognizer()
    full_text = ""
    for i, chunk in enumerate(chunks, 1):
        with sr.AudioFile(chunk) as source:
            audio_data = recognizer.record(source)
            try:
                print(f"Recognizing chunk {i}/{len(chunks)}...")
                text = recognizer.recognize_sphinx(audio_data)
                full_text += text + "\n"
            except sr.UnknownValueError:
                print(f"Chunk {i}: Speech recognition could not understand audio")
            except sr.RequestError as e:
                print(f"Chunk {i}: Could not request results from speech recognition service; {e}")
    return full_text

if __name__ == "__main__":
    video_url = 'https://rumble.com/v12esw3-warroom-battle-ground-ep-36-scotus-hears-praying-football-coach-case.html'  # Replace this with the video URL you want to download and convert
    video_path = "downloaded_video.mp4"
    audio_path = "audio.wav"

    # Download video
    download_video(video_url, video_path)

    # Extract audio from video
    extract_audio(video_path, audio_path)

    # Split audio into chunks
    chunks = split_audio(audio_path)

    # Perform speech-to-text conversion on each chunk and track progress
    full_transcript = recognize_audio_chunks(chunks)
    print(full_transcript)


In [ ]:
import os
import glob

def delete_audio_files(audio_path_pattern="audio.wav", chunk_file_pattern="chunk*.wav"):
    # Delete the main audio file(s)
    for audio_file in glob.glob(audio_path_pattern):
        os.remove(audio_file)
        print(f"Deleted audio file: {audio_file}")
    
    # Delete all chunk files
    for chunk_file in glob.glob(chunk_file_pattern):
        os.remove(chunk_file)
        print(f"Deleted chunk file: {chunk_file}")

# Example usage
delete_audio_files()
